# Expanding the slow SSM in Kolmogorov flow

We solve the vorticity equation

$$
\frac{\partial \omega }{\partial t} = \nu \Delta \omega - (u\cdot \nabla)\omega + F.
$$

The laminar solution is 

$$
\omega_0 = \frac{1}{4\nu}\cos(4 y) 
$$

with 
$$
\frac{\partial \omega_0}{\partial t} = 0
$$

We solve following ordinary differential equations in Fourier-space

$$
    \frac{d\hat{\omega}}{dt} = -\frac{1}{\text{Re}}(k_x^2 + k_y^2) \hat{\omega} - \widehat{(\mathbf{u}\cdot \nabla )\omega} - 4 \delta_{k_y, 4} \delta_{k_x,0},
$$
for the discrete wave numbers $k_x,k_y=-12, ..., 12$, resulting in a total of $576$ degrees of freedom. The nonlinearity is evaluated by introducing the stream function $\psi(x,y)$ as 
$$
\mathbf{u} = \begin{pmatrix} \partial_y \psi  \\ -\partial_x \psi \end{pmatrix}.
$$
The streamfunction can be recovered by solving the Poisson equation in Fourier space
\begin{equation}
     (k_x^2 + k_y^2)\hat{\psi} = -\hat{\omega}.
\end{equation}
The inverse Fourier transform of $\hat{\psi}$ then allows us to evaluate the nonlinearity $(\mathbf{u}\cdot \nabla )\omega$ in real space and take the Fourier transform subsequently. For dealiasing, we use the 3/2 scheme


The spectral simulation is based on the matlab code of Mohammad Farazmand. See __M. Farazmand, A. K. Saibaba (2023) Tensor-based flow reconstruction from optimally located sensor measurements. JFM 962__

In [39]:
import numpy as np
import torch
from kolmogorov import gvars, rhs_torch_fourier_tf, newton_method
from scipy.integrate import solve_ivp
from tqdm import tqdm
import hdf5storage
L1 = 2*np.pi
L2 = 2*np.pi 
n1 = 24
n2 = 24

x, y, kx, ky = gvars(n1,n2,L1,L2)
x = torch.tensor(x, dtype = torch.float64, device="cpu").requires_grad_(True)
y = torch.tensor(y, dtype = torch.float64,device="cpu").requires_grad_(True)
kx = torch.tensor(kx, dtype = torch.float64, device="cpu").requires_grad_(True)
ky = torch.tensor(ky, dtype = torch.float64, device="cpu").requires_grad_(True)


## Heteroclinic orbit

The laminar solution becomes unstable at around $Re=9.1$, after which a pair of new stable fixed points and the saddle-type laminar exist. 
Here we construct the slow spectral submanifold (SSM) of one of the new fixed points, denoted $\omega_1$. To find this fixed point, we first integrate forward along the unstable manifold of the laminar state at $Re=11$. 

In [3]:
Re = 11

In [4]:
nu = 1/Re
w_0 = -Re * torch.cos(4 * y)/4
w_0_ft = torch.fft.fftn(w_0).ravel()
w_0_ft.retain_grad()
rh = lambda x : rhs_torch_fourier_tf(x, kx, ky, nu, x, y, n1, n2)

In [6]:
jac = torch.autograd.functional.jacobian(rh, w_0_ft)
s, v = torch.linalg.eig(jac)
ss = s.numpy()
lamb = np.max(ss.real)
eig_index = np.argmax(ss.real)
print(Re, lamb,)


11 0.040500138545508116


In [7]:
def ode_rhs(t, x):
        w_flat = torch.tensor(x)
        return rhs_torch_fourier_tf(w_flat, kx, ky, nu, x, y, n1, n2).detach().numpy()

We integrate forward in time until $\lambda t \approx 20$

In [10]:
time_length = 20 / lamb
teval = np.linspace(0, time_length, 5000)
ic_plus = (w_0_ft + 1e-1 * w_1).detach()
ic_minus =(w_0_ft - 1e-1 * w_1).detach()
plus = solve_ivp(ode_rhs, [0,time_length], ic_plus, t_eval=teval).y
minus = solve_ivp(ode_rhs, [0,time_length], ic_minus, t_eval=teval).y

## Nontrivial fixed point

In [28]:
# find the fixed point
print('Newton method starts')
sol = newton_method(rh, torch.tensor(minus[:,-1]))
print('Newton method ends')
jac = torch.autograd.functional.jacobian(rh, sol['x'])
s, v = torch.linalg.eig(jac)
sortedEig, indices=torch.sort(s.real, dim=0, descending=True)
w_0orted = v[:, indices]
lamb = s[indices][1]
w_1 = w_0orted[:, 1] #exclude zero eigenvalue    
## coeffs for ssmtool:


Newton method starts
Newton method ends


In [25]:
sol['success']b

1

We also save the left eigenvalue

In [32]:
s_left, v_left = torch.linalg.eig(torch.conj(jac).T)
sortedEig, indices=torch.sort(s_left.real, dim=0, descending=True)
w_0orted_left = v_left[:, indices]
lamb = s_left[indices][1]
w_1_left = w_0orted_left[:, 1] #exclude zero eigenvalue    
## coeffs for ssmtool:

In [19]:
ss_left = s_left.numpy()
lamb = np.max(ss_left.real)
eig_index_left = np.argmax(ss_left.real)

In [21]:
w_1_left = v_left[:,eig_index_left].real


We use [SSMTool](https://github.com/haller-group/SSMTool-2.4) for the subsequent computations, which requires the nonlinear coefficients of the RHS of the ODE expressed around the $\omega_1$ fixed point, i.e. the constant term must be zero. We obtain these coefficients by Taylor-expanding using automatic differentiation. Since the vorticity equation is quadratic, it suffices to compute the Jacobian and the second derivative-tensor. 

In [37]:
jac_fn = lambda x : torch.autograd.functional.jacobian(rh, x, create_graph = True)
hess = []
for ii in tqdm(range(576)):
    rhs_component_i = lambda x : rh(x)[ii]
    jac_fn = lambda x : torch.autograd.functional.jacobian(rhs_component_i, x, create_graph = True)
    hess.append(torch.autograd.functional.jacobian(jac_fn, sol['x']))
hh = np.array([h.numpy() for h in hess])

100%|█████████████████████████████████████████| 576/576 [02:22<00:00,  4.05it/s]


In [40]:
to_matlab = {}
to_matlab['hessian'] = hh 
to_matlab['jacobian'] = np.array(jac)
to_matlab['w0'] = sol['x'].detach().numpy()
to_matlab['w1'] = w_1.detach().numpy()
to_matlab['plus'] = plus
to_matlab['minus'] = minus
to_matlab['lamb'] = lamb.item().real
to_matlab['w_1_left'] = w_1_left.detach().numpy()
hdf5storage.write(to_matlab,  filename = "Kolmogorov_coefficients_around_fp.mat", matlab_compatible=True)
